In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import psychoanalyze as pa

In [2]:
min_intensity = -4
max_intensity = 4
k=1
n_trials = 100
n_blocks = 2

In [3]:
intensity_choices = pd.Index(
    range(min_intensity, max_intensity + 1), name="Intensity"
)
intensity_choices

RangeIndex(start=-4, stop=5, step=1, name='Intensity')

In [6]:

hit_rates = pa.blocks.model_hit_rates(intensity_choices, k)
trials = pa.trials.moc_sample(intensity_choices, n_trials, k, n_blocks=2)
trials

Intensity  Result
Block                      
0     0           0   False
      1           1   False
      2           1    True
      3           4    True
      4           2    True
...             ...     ...
1     95          4    True
      96          3    True
      97          3    True
      98          2    True
      99          4    True

[200 rows x 2 columns]

In [9]:
observed = pa.points.from_trials(trials)
observed.reset_index()["Intensity"].unique()

array([-4, -3, -2, -1,  0,  1,  2,  3,  4])

In [19]:

fits = trials.groupby("Block").apply(pa.blocks.get_fit)
fits

TypeError: 'numpy.float64' object is not iterable

In [29]:
pa.blocks.get_fit(trials)

{'slope': array([0.96832043]), 'intercept': array([-0.01797368])}

In [34]:
trials.groupby("Block").apply(pa.blocks.get_fit).droplevel(-1)

,slope,intercept
Block,,
0,1.075417,-0.095133
1,0.856080,0.045637


array([-0.01797368])

In [ ]:

observed_points1 = pa.points.from_trials(trials1)
fits1 = pa.blocks.get_fit(trials1)
predictions1 = pa.blocks.make_predictions(fits1, intensity_choices)

trials2 = pa.trials.moc_sample(intensity_choices, n_trials, k)
observed_points2 = pa.points.from_trials(trials2)
fits2 = pa.blocks.get_fit(trials2)
predictions2 = pa.blocks.make_predictions(fits2, intensity_choices)


In [69]:

intensity_choices = pd.Index(
    range(min_intensity, max_intensity + 1), name="Intensity"
)

hit_rates = pa.blocks.model_hit_rates(intensity_choices, k)

trials = [
    pa.trials.moc_sample(intensity_choices, n_trials, k) for _ in range(n_blocks)
]

trials = pd.concat({block:pa.trials.moc_sample(intensity_choices, n_trials, k) for block in range(n_blocks)}, names=["Block", "TrialID"])
trials


Intensity  Result
Block TrialID                   
0     0                4    True
      1               -3   False
      2                0   False
      3                3    True
      4               -4   False
...                  ...     ...
1     95               2    True
      96              -1    True
      97              -3   False
      98               4    True
      99               4    True

[200 rows x 2 columns]

In [25]:
predictions1

,Hit Rate
Intensity,
-4,0.006433
-3,0.022315
-2,0.074472
-1,0.220978
0,0.500000
1,0.779022
2,0.925528
3,0.977685
4,0.993567


In [26]:
predictions = pd.concat(
    [predictions1, predictions2], keys=["1", "2"], names=["Block"]
)
predictions

Hit Rate
Block Intensity          
1     -4         0.006433
      -3         0.022315
      -2         0.074472
      -1         0.220978
       0         0.500000
       1         0.779022
       2         0.925528
       3         0.977685
       4         0.993567
2     -4         0.036053
      -3         0.078382
      -2         0.162054
      -1         0.305442
       0         0.500000
       1         0.694558
       2         0.837946
       3         0.921618
       4         0.963947

In [21]:
observed = pd.concat([observed_points1, observed_points2], keys=["1", "2"], names=["Block"])
observed

Hit Rate   n
Block Intensity              
1     -4         0.000000  12
      -3         0.000000  12
      -2         0.000000   8
      -1         0.166667  12
       0         0.333333   3
       1         0.789474  19
       2         1.000000   6
       3         0.928571  14
       4         0.928571  14
2     -4         0.000000  11
      -3         0.083333  12
      -2         0.153846  13
      -1         0.357143  14
       0         0.166667  12
       1         0.800000  10
       2         0.666667   6
       3         0.888889   9
       4         1.000000  13

In [37]:
hit_rates = pd.Series(
    pa.blocks.model_hit_rates(intensity_choices, k),
    name="Hit Rate",
    index=pd.Index(intensity_choices, name="Intensity"),
)

hit_rates = hit_rates.reset_index()
hit_rates["Block"] = "Prior"
hit_rates = hit_rates.set_index(["Block", "Intensity"])
hit_rates

Hit Rate
Block Intensity          
Prior -4         0.017986
      -3         0.047426
      -2         0.119203
      -1         0.268941
       0         0.500000
       1         0.731059
       2         0.880797
       3         0.952574
       4         0.982014

In [55]:
import random
intensity_choices = pd.Index(range(-4, 5), name="Intensity")
[random.choice(intensity_choices) for _ in range(n_trials)]
# intensity_choices

[-2,
 3,
 4,
 4,
 -4,
 -4,
 4,
 0,
 4,
 -1,
 0,
 0,
 4,
 -2,
 2,
 -3,
 -1,
 2,
 1,
 -3,
 -3,
 3,
 1,
 -3,
 4,
 4,
 3,
 3,
 3,
 4,
 -2,
 1,
 1,
 3,
 2,
 0,
 -2,
 3,
 2,
 -1,
 4,
 4,
 -1,
 4,
 3,
 -1,
 -4,
 -3,
 -1,
 4,
 -3,
 3,
 -4,
 0,
 4,
 -4,
 -4,
 -1,
 3,
 -1,
 -2,
 1,
 3,
 2,
 -3,
 0,
 1,
 4,
 1,
 0,
 2,
 -3,
 -4,
 -2,
 3,
 2,
 2,
 4,
 1,
 -4,
 4,
 -2,
 -1,
 -3,
 -2,
 2,
 -2,
 1,
 -3,
 0,
 4,
 0,
 -3,
 3,
 2,
 3,
 -2,
 4,
 4,
 1]

In [53]:
pd.Series([1 / (1 + np.exp(-k * x)) for x in intensity_choices], index=intensity_choices, name="Hit Rate")

Intensity
-4    0.017986
-3    0.047426
-2    0.119203
-1    0.268941
0     0.500000
1     0.731059
2     0.880797
3     0.952574
4     0.982014
Name: Hit Rate, dtype: float64

In [59]:
intensity_choices = pd.Index(
    range(min_intensity, max_intensity + 1), name="Intensity"
)

hit_rates = pa.blocks.model_hit_rates(intensity_choices, k)
hit_rates

Intensity
-4    0.017986
-3    0.047426
-2    0.119203
-1    0.268941
0     0.500000
1     0.731059
2     0.880797
3     0.952574
4     0.982014
Name: Hit Rate, dtype: float64

Float64Index([0.01798620996209156, 0.04742587317756678, 0.11920292202211755,
               0.2689414213699951,                 0.5,  0.7310585786300049,
               0.8807970779778823,  0.9525741268224334,  0.9820137900379085],
             dtype='float64', name='Intensity')

In [42]:
1 / (1 + np.exp(-k * pd.Index(list(range(min_intensity, max_intensity + 1)), name="Intensity")))

Float64Index([0.01798620996209156, 0.04742587317756678, 0.11920292202211755,
               0.2689414213699951,                 0.5,  0.7310585786300049,
               0.8807970779778823,  0.9525741268224334,  0.9820137900379085],
             dtype='float64', name='Intensity')

In [71]:
pd.DataFrame({
    "U": {
        "k": [1,2,3]
    }
})

,U
k,"[1, 2, 3]"
